In [ ]:
# notebooks/regression.ipynb
import sys
sys.path.append('../scripts')

from mongo_connection import get_matches_collection

collection = get_matches_collection()

result = collection.find_one({"home_team": "Romania", "away_team": "Greece"})
print(result)